In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [34]:
confirm_df = pd.read_csv('2020-03-21/covid_confirmed_daily_updates.csv')
#confirm_df.head()

In [26]:
recover_df = pd.read_csv('2020-03-20/covid_recovered_daily_updates.csv')
#recover_df.head()

In [27]:
death_df = pd.read_csv('2020-03-20/covid_deaths_daily_updates.csv')
#death_df.head()

In [6]:
def cal_moving_avg(input_df, window=2):
    """Calculates Moving avg using the window size."""
    _df = input_df.rolling(window, axis=1).median()['3/16/20']
    
    return _df

def cal_ewma(input_df, comm=0.3):
    """Calculates the exp wighted moving average using the window size."""
    _df = input_df.ewm(com=comm).mean()['3/16/20']
    
    return _df

In [7]:
all_pred_df = pd.DataFrame()
all_pred_df_ew = pd.DataFrame()

for df in [confirm_df, recover_df, death_df]:
    _df = cal_moving_avg(df.groupby(['Country/Region']).sum(), window=2)
    ew_df = cal_ewma(df.groupby(['Country/Region']).sum(), comm=0.4)
    all_pred_df = pd.concat([all_pred_df, _df], axis=1)
    all_pred_df_ew = pd.concat([all_pred_df_ew, ew_df], axis=1)
    
all_pred_df.columns =['Confirmed', 'Recovered', 'Death']    
all_pred_df_ew.columns =['Confirmed', 'Recovered', 'Death']  

C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  import sys
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


## Exponential Smoothing

In [5]:
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_squared_error, mean_absolute_error
from pandas import read_csv
from numpy import array

In [6]:
# one-step Holt Winter’s Exponential Smoothing forecast
def exp_smoothing_forecast(history, config):
	t,d,s,p,b,r = config
	# define model
	history = array(history)
	model = ExponentialSmoothing(history, trend=t, damped=d, seasonal=s, seasonal_periods=p)
	# fit model
	model_fit = model.fit(optimized=True, use_boxcox=b, remove_bias=r)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

In [7]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

In [8]:
# root mean squared error or rmse
def measure_mae(actual, predicted):
	return mean_absolute_error(actual, predicted)

In [9]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

In [10]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = exp_smoothing_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_mae(test, predictions)
	return error

In [11]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
#	print(cfg)    
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (cfg, result)

In [12]:
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

In [13]:
# create a set of exponential smoothing configs to try
def exp_smoothing_configs(seasonal=[None]):
	models = list()
	# define config lists
	t_params = ['add', 'mul', None]
	d_params = [True, False]
	s_params = ['add', 'mul', None]
	p_params = seasonal
	b_params = [True, False]
	r_params = [True, False]
	# create config instances
	for t in t_params:
		for d in d_params:
			for s in s_params:
				for p in p_params:
					for b in b_params:
						for r in r_params:
							cfg = [t,d,s,p,b,r]
							models.append(cfg)
	return models

In [31]:
conf_country = []
conf_tags = []
conf_error = []
conf_pred = []
conf_pred_1 = []
n_test = 7
cfg_list = exp_smoothing_configs()
for i in range(66,67):#np.shape(confirm_df)[0]):
    print(confirm_df.iloc[i][0])
    data = confirm_df.iloc[i][1:].values
    scores = grid_search(data, cfg_list, n_test,parallel=False)
    conf_tags.append(scores[:1][0][0])
    conf_error.append(scores[:1][0][1])
    conf_country.append(confirm_df.iloc[i][0])
    conf_pred.append(exp_smoothing_forecast(data.astype(float),scores[:1][0][0]))
    conf_pred_1.append(exp_smoothing_forecast(data[:-1].astype(float),scores[:1][0][0]))
    

India
 > Model[['add', True, None, None, False, True]] 8.090
 > Model[['add', True, None, None, False, False]] 8.574
 > Model[['add', False, None, None, False, True]] 8.091
 > Model[['add', False, None, None, False, False]] 8.574
 > Model[[None, False, None, None, False, True]] 15.258
 > Model[[None, False, None, None, False, False]] 17.286


In [106]:
df = pd.DataFrame(zip(conf_country,conf_pred))

df.columns = ['Country/Region','Confirmed']

In [161]:
df_conf_new = pd.DataFrame(zip(conf_country,conf_tags,conf_error,conf_pred,conf_pred_1))

df_conf_new.columns = ['Country/Region','Tags','mae_error','conf_predicted','conf_past_pred']

In [162]:
df_conf_new.to_csv('2020-03-19/conf_result.csv')

In [22]:

error_test = []
test_1 = []

for j in range(1,11):
    print("doing for {}".format(j))
    n_test = j
    conf_country = []
    conf_tags = []
    conf_error = []
    conf_pred = []
    conf_pred_1 = []

    cfg_list = exp_smoothing_configs()
    for i in range(np.shape(confirm_df)[0]):
        print(confirm_df.iloc[i][0])
        data = confirm_df.iloc[i][1:].values
        scores = grid_search(data, cfg_list, n_test,parallel=False)
        conf_tags.append(scores[:1][0][0])
        conf_error.append(scores[:1][0][1])
        conf_country.append(confirm_df.iloc[i][0])
        conf_pred.append(exp_smoothing_forecast(data.astype(float),scores[:1][0][0]))
        conf_pred_1.append(exp_smoothing_forecast(data[:-1].astype(float),scores[:1][0][0]))



    df_conf_new = pd.DataFrame(zip(conf_country,conf_tags,conf_error,conf_pred,conf_pred_1))

    df_conf_new.columns = ['Country/Region','Tags','mae_error','conf_predicted','conf_past_pred']

    test = pd.merge(confirm_df,df_conf_new[['Country/Region','conf_predicted','conf_past_pred']],how='inner',on='Country/Region')[['3/18/20','conf_predicted','Country/Region']]

    error_test.append(measure_mae(test['3/18/20'],test['conf_predicted'].fillna(0)))
    test_1.append(j)

doing for 1
Afghanistan
 > Model[['add', True, None, None, False, True]] 0.794
 > Model[['add', True, None, None, False, False]] 0.642
 > Model[['add', False, None, None, False, True]] 2.831
 > Model[['add', False, None, None, False, False]] 2.689
 > Model[[None, False, None, None, False, True]] 0.386
 > Model[[None, False, None, None, False, False]] 0.000
Albania
 > Model[['add', True, None, None, False, True]] 1.808
 > Model[['add', True, None, None, False, False]] 1.578
 > Model[['add', False, None, None, False, True]] 2.166
 > Model[['add', False, None, None, False, False]] 1.995
 > Model[[None, False, None, None, False, True]] 3.035
 > Model[[None, False, None, None, False, False]] 4.000
Algeria
 > Model[['add', True, None, None, False, True]] 8.223
 > Model[['add', True, None, None, False, False]] 8.510
 > Model[['add', False, None, None, False, True]] 7.029
 > Model[['add', False, None, None, False, False]] 7.223
 > Model[[None, False, None, None, False, True]] 12.948
 > Model[[

 > Model[['add', True, None, None, False, True]] 6.107
 > Model[['add', True, None, None, False, False]] 6.000
 > Model[['add', False, None, None, False, True]] 6.107
 > Model[['add', False, None, None, False, False]] 6.000
 > Model[[None, False, None, None, False, True]] 0.175
 > Model[[None, False, None, None, False, False]] 0.000
Canada
 > Model[['add', True, None, None, False, True]] 74.775
 > Model[['add', True, None, None, False, False]] 78.640
 > Model[['add', False, None, None, False, True]] 74.775
 > Model[['add', False, None, None, False, False]] 78.640
 > Model[[None, False, None, None, False, True]] 170.617
 > Model[[None, False, None, None, False, False]] 179.000
Central African Republic
 > Model[['add', True, None, None, False, True]] 0.018
 > Model[['add', True, None, None, False, False]] 0.000
 > Model[['add', False, None, None, False, True]] 0.018
 > Model[['add', False, None, None, False, False]] 0.018
 > Model[[None, False, None, None, False, True]] 0.018
 > Model[[N

C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


Dominican Republic
 > Model[['add', True, None, None, False, True]] 0.193
 > Model[['add', True, None, None, False, False]] 0.045
 > Model[['add', False, None, None, False, True]] 0.193
 > Model[['add', False, None, None, False, False]] 0.045
 > Model[[None, False, None, None, False, True]] 0.368
 > Model[[None, False, None, None, False, False]] 0.000
Ecuador
 > Model[['add', True, None, None, False, True]] 41.763
 > Model[['add', True, None, None, False, False]] 42.253
 > Model[['add', False, None, None, False, True]] 41.763
 > Model[['add', False, None, None, False, False]] 42.253
 > Model[[None, False, None, None, False, True]] 51.983
 > Model[[None, False, None, None, False, False]] 53.000
Egypt
 > Model[['add', True, None, None, False, True]] 35.302
 > Model[['add', True, None, None, False, False]] 34.007
 > Model[['add', False, None, None, False, True]] 35.302
 > Model[['add', False, None, None, False, False]] 34.007
 > Model[[None, False, None, None, False, True]] 3.438
 > Model

Indonesia
 > Model[['add', True, None, None, False, True]] 29.857
 > Model[['add', True, None, None, False, False]] 30.932
 > Model[['add', False, None, None, False, True]] 29.857
 > Model[['add', False, None, None, False, False]] 30.932
 > Model[[None, False, None, None, False, True]] 51.983
 > Model[[None, False, None, None, False, False]] 55.000
Iran
 > Model[['add', True, None, None, False, True]] 7.036
 > Model[['add', True, None, None, False, False]] 14.000
 > Model[['add', False, None, None, False, True]] 7.036
 > Model[['add', False, None, None, False, False]] 14.000
 > Model[[None, False, None, None, False, True]] 908.424
 > Model[[None, False, None, None, False, False]] 1192.000
Iraq
 > Model[['add', True, None, None, False, True]] 2.942
 > Model[['add', True, None, None, False, False]] 4.086
 > Model[['add', False, None, None, False, True]] 2.790
 > Model[['add', False, None, None, False, False]] 3.917
 > Model[[None, False, None, None, False, True]] 7.299
 > Model[[None, Fa

...

Mexico
 > Model[['add', True, None, None, False, True]] 6.177
 > Model[['add', True, None, None, False, False]] 6.141
 > Model[['add', False, None, None, False, True]] 6.293
 > Model[['add', False, None, None, False, False]] 6.260
 > Model[[None, False, None, None, False, True]] 8.239
 > Model[[None, False, None, None, False, False]] 8.600
Moldova
 > Model[['add', True, None, None, False, True]] 4.129
 > Model[['add', True, None, None, False, False]] 4.122
 > Model[['add', False, None, None, False, True]] 3.743
 > Model[['add', False, None, None, False, False]] 3.734
 > Model[[None, False, None, None, False, True]] 3.022
 > Model[[None, False, None, None, False, False]] 2.998
Monaco
 > Model[['add', True, None, None, False, True]] 0.878
 > Model[['add', True, None, None, False, False]] 0.874
 > Model[['add', False, None, None, False, True]] 0.878
 > Model[['add', False, None, None, False, False]] 0.878
 > Model[[None, False, None, None, False, True]] 0.637
 > Model[[None, False, None, 

 > Model[['add', True, None, None, False, True]] 0.823
 > Model[['add', True, None, None, False, False]] 0.825
 > Model[['add', False, None, None, False, True]] 0.820
 > Model[['add', False, None, None, False, False]] 0.819
 > Model[[None, False, None, None, False, True]] 0.875
 > Model[[None, False, None, None, False, False]] 0.898
Saint Lucia
 > Model[['add', True, None, None, False, True]] 0.404
 > Model[['add', True, None, None, False, False]] 0.398
 > Model[['add', False, None, None, False, True]] 0.459
 > Model[['add', False, None, None, False, False]] 0.452
 > Model[[None, False, None, None, False, True]] 0.309
 > Model[[None, False, None, None, False, False]] 0.298
Saint Vincent and the Grenadines
 > Model[['add', True, None, None, False, True]] 0.204
 > Model[['add', True, None, None, False, False]] 0.198
 > Model[['add', False, None, None, False, True]] 0.201
 > Model[['add', False, None, None, False, False]] 0.200
 > Model[[None, False, None, None, False, True]] 0.203
 > Mod

C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:70: RuntimeWarning: divide by zero encountered in double_scalars
  b[i] = (beta * (l[i] / l[i - 1])) + (betac * b[i - 1]**phi)
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:70: RuntimeWarning: invalid value encountered in double_scalars
  b[i] = (beta * (l[i] / l[i - 1])) + (betac * b[i - 1]**phi)
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:69: RuntimeWarning: invalid value encountered in double_scalars
  l[i] = (y_alpha[i - 1]) + (alphac * (l[i - 1] * b[i - 1]**phi))


Tanzania
 > Model[['add', True, None, None, False, True]] 0.396
 > Model[['add', True, None, None, False, False]] 0.398
 > Model[['add', False, None, None, False, True]] 0.394
 > Model[['add', False, None, None, False, False]] 0.393
 > Model[[None, False, None, None, False, True]] 0.396
 > Model[[None, False, None, None, False, False]] 0.398
Thailand
 > Model[['add', True, None, None, True, True]] 8.819
 > Model[['add', True, None, None, True, False]] 8.901
 > Model[['add', True, None, None, False, True]] 7.804
 > Model[['add', True, None, None, False, False]] 7.955
 > Model[['add', False, None, None, True, True]] 8.714
 > Model[['add', False, None, None, True, False]] 8.795
 > Model[['add', False, None, None, False, True]] 7.681
 > Model[['add', False, None, None, False, False]] 7.825
 > Model[['mul', True, None, None, True, True]] 7.772
 > Model[['mul', True, None, None, True, False]] 8.319
 > Model[['mul', True, None, None, False, True]] 8.405
 > Model[['mul', True, None, None, Fals

C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


In [24]:
error_test

[111.19289379195011,
 110.90017973263524,
 111.29523743764348,
 116.96013696967906,
 112.0901075695758,
 112.20034694689498,
 111.71106848368153,
 116.95282836005754,
 117.0382943920954,
 116.98560499253304]

In [23]:
test_1

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [16]:
test

,3/18/20,conf_predicted,Country/Region
0,22,NaN,Afghanistan
1,59,NaN,Albania
2,74,NaN,Algeria
3,39,NaN,Andorra
4,1,NaN,Antigua and Barbuda
5,79,NaN,Argentina
6,84,NaN,Armenia
7,568,NaN,Australia
8,1646,NaN,Austria
9,28,NaN,Azerbaijan


In [155]:
test['more_past'] = test['conf_past_pred'] > test['3/17/20'] + 5
test['more'] = test['conf_predicted'] > test['3/18/20'] + 5

In [158]:
measure_mae(test['3/18/20'],test['conf_predicted'].fillna(0))

35.25890291655596

In [160]:
test.loc[test['3/17/20']>1000]

,3/17/20,conf_past_pred,3/18/20,conf_predicted,Country/Region,more_past,more
8,1332,1199.315327,1646,1570.933789,Austria,False,False
14,1243,1247.875291,1486,1430.848358,Belgium,False,False
38,1024,990.036364,1115,1117.642857,Denmark,False,False
48,7699,7261.725655,9105,8791.426749,France,False,False
52,9257,8632.972857,12327,10966.090069,Germany,False,False
65,16169,16063.145455,17361,17368.035714,Iran,False,True
69,31506,31874.406745,35713,35168.707462,Italy,True,False
75,8320,8299.279625,8413,8382.400563,"Korea, South",False,False
85,81058,81061.046992,81102,81084.772778,Mainland China,False,False
100,1711,1630.214447,2058,1976.316540,Netherlands,False,False


In [107]:
rec_country = []
rec_tags = []
rec_error = []
rec_pred = []
rec_pred_1 = []
n_test = 7
cfg_list = exp_smoothing_configs()
for i in range(np.shape(recover_df)[0]):
    print(recover_df.iloc[i][0])
    data = recover_df.iloc[i][1:].values
    scores = grid_search(data, cfg_list, n_test,parallel=False)
    rec_tags.append(scores[:1][0][0])
    rec_error.append(scores[:1][0][1])
    rec_country.append(recover_df.iloc[i][0])
    rec_pred.append(exp_smoothing_forecast(data.astype(float),scores[:1][0][0]))
    rec_pred_1.append(exp_smoothing_forecast(data[:-1].astype(float),scores[:1][0][0]))
    

Afghanistan
 > Model[['add', True, None, None, False, True]] 0.286
 > Model[['add', True, None, None, False, False]] 0.283
 > Model[['add', False, None, None, False, True]] 0.282
 > Model[['add', False, None, None, False, False]] 0.281
 > Model[[None, False, None, None, False, True]] 0.286
 > Model[[None, False, None, None, False, False]] 0.283
Albania
 > Model[['add', True, None, None, False, True]] 0.000
 > Model[['add', True, None, None, False, False]] 0.000
 > Model[['add', False, None, None, False, True]] 0.000
 > Model[['add', False, None, None, False, False]] 0.000
 > Model[[None, False, None, None, False, True]] 0.000
 > Model[[None, False, None, None, False, False]] 0.000


C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


Algeria
 > Model[['add', True, None, None, False, True]] 3.617
 > Model[['add', True, None, None, False, False]] 3.557
 > Model[['add', False, None, None, False, True]] 3.986
 > Model[['add', False, None, None, False, False]] 3.889
 > Model[[None, False, None, None, False, True]] 2.937
 > Model[[None, False, None, None, False, False]] 2.835
Andorra
 > Model[['add', True, None, None, False, True]] 0.412
 > Model[['add', True, None, None, False, False]] 0.405
 > Model[['add', False, None, None, False, True]] 0.412
 > Model[['add', False, None, None, False, False]] 0.405
 > Model[[None, False, None, None, False, True]] 0.402
 > Model[[None, False, None, None, False, False]] 0.420
Antigua and Barbuda
 > Model[['add', True, None, None, False, True]] 0.000
 > Model[['add', True, None, None, False, False]] 0.000
 > Model[['add', False, None, None, False, True]] 0.000
 > Model[['add', False, None, None, False, False]] 0.000
 > Model[[None, False, None, None, False, True]] 0.000
 > Model[[None,

C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


Argentina
 > Model[['add', True, None, None, False, True]] 0.591
 > Model[['add', True, None, None, False, False]] 0.585
 > Model[['add', False, None, None, False, True]] 0.587
 > Model[['add', False, None, None, False, False]] 0.580
 > Model[[None, False, None, None, False, True]] 0.576
 > Model[[None, False, None, None, False, False]] 0.569
Armenia
 > Model[['add', True, None, None, False, True]] 0.283
 > Model[['add', True, None, None, False, False]] 0.283
 > Model[['add', False, None, None, False, True]] 0.279
 > Model[['add', False, None, None, False, False]] 0.278
 > Model[[None, False, None, None, False, True]] 0.283
 > Model[[None, False, None, None, False, False]] 0.283
Australia
 > Model[['add', True, None, None, False, True]] 0.594
 > Model[['add', True, None, None, False, False]] 0.594
 > Model[['add', False, None, None, False, True]] 0.594
 > Model[['add', False, None, None, False, False]] 0.594
 > Model[[None, False, None, None, False, True]] 0.586
 > Model[[None, False, 

...

C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


Venezuela
 > Model[['add', True, None, None, False, True]] 0.000
 > Model[['add', True, None, None, False, False]] 0.000
 > Model[['add', False, None, None, False, True]] 0.000
 > Model[['add', False, None, None, False, False]] 0.000
 > Model[[None, False, None, None, False, True]] 0.000
 > Model[[None, False, None, None, False, False]] 0.000


C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


Vietnam
 > Model[['add', True, None, None, False, True]] 0.313
 > Model[['add', True, None, None, False, False]] 0.313
 > Model[['add', False, None, None, False, True]] 0.324
 > Model[['add', False, None, None, False, False]] 0.325
 > Model[[None, False, None, None, False, True]] 0.297
 > Model[[None, False, None, None, False, False]] 0.000
Zambia
 > Model[['add', True, None, None, False, True]] 0.000
 > Model[['add', True, None, None, False, False]] 0.000
 > Model[['add', False, None, None, False, True]] 0.000
 > Model[['add', False, None, None, False, False]] 0.000
 > Model[[None, False, None, None, False, True]] 0.000
 > Model[[None, False, None, None, False, False]] 0.000


C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


In [108]:
df_1 = pd.DataFrame(zip(rec_country,rec_pred))

df_1.columns = ['Country/Region','Recovered']

df_rec_new = pd.DataFrame(zip(rec_country,rec_tags,rec_error,rec_pred,rec_pred_1))

df_rec_new.columns = ['Country/Region','Tags','mae_error','rec_predicted','rec_past_pred']

In [164]:
df_rec_new.to_csv('2020-03-19/rec_result.csv')

In [109]:
de_country = []
de_tags = []
de_error = []
de_pred = []
de_pred_1 = []
n_test = 7
cfg_list = exp_smoothing_configs()
for i in range(np.shape(death_df)[0]):
    print(death_df.iloc[i][0])
    data = death_df.iloc[i][1:].values
    scores = grid_search(data, cfg_list, n_test,parallel=False)
    de_tags.append(scores[:1][0][0])
    de_error.append(scores[:1][0][1])
    de_country.append(death_df.iloc[i][0])
    de_pred.append(exp_smoothing_forecast(data.astype(float),scores[:1][0][0]))
    de_pred_1.append(exp_smoothing_forecast(data[:-1].astype(float),scores[:1][0][0]))
    

Afghanistan
 > Model[['add', True, None, None, False, True]] 0.000
 > Model[['add', True, None, None, False, False]] 0.000
 > Model[['add', False, None, None, False, True]] 0.000
 > Model[['add', False, None, None, False, False]] 0.000
 > Model[[None, False, None, None, False, True]] 0.000
 > Model[[None, False, None, None, False, False]] 0.000


C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


Albania
 > Model[['add', True, None, None, False, True]] 0.294
 > Model[['add', True, None, None, False, False]] 0.283
 > Model[['add', False, None, None, False, True]] 0.290
 > Model[['add', False, None, None, False, False]] 0.288
 > Model[[None, False, None, None, False, True]] 0.294
 > Model[[None, False, None, None, False, False]] 0.283
Algeria
 > Model[['add', True, None, None, False, True]] 1.005
 > Model[['add', True, None, None, False, False]] 0.997
 > Model[['add', False, None, None, False, True]] 1.001
 > Model[['add', False, None, None, False, False]] 0.992
 > Model[[None, False, None, None, False, True]] 1.137
 > Model[[None, False, None, None, False, False]] 1.140
Andorra
 > Model[['add', True, None, None, False, True]] 0.000
 > Model[['add', True, None, None, False, False]] 0.000
 > Model[['add', False, None, None, False, True]] 0.000
 > Model[['add', False, None, None, False, False]] 0.000
 > Model[[None, False, None, None, False, True]] 0.000
 > Model[[None, False, None

C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


Antigua and Barbuda
 > Model[['add', True, None, None, False, True]] 0.000
 > Model[['add', True, None, None, False, False]] 0.000
 > Model[['add', False, None, None, False, True]] 0.000
 > Model[['add', False, None, None, False, False]] 0.000
 > Model[[None, False, None, None, False, True]] 0.000
 > Model[[None, False, None, None, False, False]] 0.000


...

C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


Vietnam
 > Model[['add', True, None, None, False, True]] 0.000
 > Model[['add', True, None, None, False, False]] 0.000
 > Model[['add', False, None, None, False, True]] 0.000
 > Model[['add', False, None, None, False, False]] 0.000
 > Model[[None, False, None, None, False, True]] 0.000
 > Model[[None, False, None, None, False, False]] 0.000


C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


Zambia
 > Model[['add', True, None, None, False, True]] 0.000
 > Model[['add', True, None, None, False, False]] 0.000
 > Model[['add', False, None, None, False, True]] 0.000
 > Model[['add', False, None, None, False, False]] 0.000
 > Model[[None, False, None, None, False, True]] 0.000
 > Model[[None, False, None, None, False, False]] 0.000


C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:734: RuntimeWarning: divide by zero encountered in log
  aic = self.nobs * np.log(sse / self.nobs) + (k) * 2
C:\Users\tushar.prasad\AppData\Local\Continuum\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters.py:736: RuntimeWarning: divide by zero encountered in log
  bic = self.nobs * np.log(sse / self.nobs) + (k) * np.log(self.nobs)


In [110]:
df_2 = pd.DataFrame(zip(de_country,de_pred))

df_2.columns = ['Country/Region','Death']

In [165]:
df_de_new = pd.DataFrame(zip(de_country,de_tags,de_error,de_pred,de_pred_1))

df_de_new.columns = ['Country/Region','Tags','mae_error','de_predicted','de_past_pred']

In [166]:
df_conf_new.to_csv('2020-03-19/de_result.csv')

In [123]:
df_total = pd.merge(df,df_1,on='Country/Region',how='outer')

df_total = pd.merge(df_total,df_2,on='Country/Region',how='outer')

In [126]:
df_total.to_excel('submission_2020_03_19.xlsx',index=False)

In [57]:
len(country)

14

In [45]:
data.astype(float)

array([  2.,   3.,   5.,   7.,   8.,   8.,  14.,  14.,  14.,  19.,  19.,
        19.,  19.,  25.,  25.,  25.,  25.,  32.,  32.,  32.,  33.,  33.,
        33.,  33.,  33.,  34.,  35.,  35.,  35.,  35.,  35.,  35.,  35.,
        35.,  37.,  40.,  40.,  41.,  42.,  42.,  43.,  43.,  43.,  47.,
        48.,  50.,  50.,  50.,  53.,  59.,  70.,  75.,  82., 114., 147.,
       177.])

In [49]:
pred.append(exp_smoothing_forecast(data.astype(float),scores[:1][0][0]))

In [47]:
pred_1.append(exp_smoothing_forecast(data[:-1].astype(float),scores[:1][0][0]))

In [48]:
exp_smoothing_forecast(data.astype(float),scores[:1][0][0])

230.95471316365447

In [33]:
exp_smoothing_forecast(data.astype(float),scores[:1][0][0])

81084.77277759151

In [30]:
scores[:1][0][0]

['mul', False, None, None, True, False]

In [28]:
country

['Afghanistan',
 'Albania',
 'Algeria',
 'Andorra',
 'Antigua and Barbuda',
 'Argentina',
 'Armenia',
 'Aruba',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Central African Republic',
 'Chile',
 'Colombia',
 'Congo (Brazzaville)',
 'Congo (Kinshasa)',
 'Costa Rica',
 "Cote d'Ivoire",
 'Croatia',
 'Cruise Ship',
 'Cuba',
 'Cyprus',
 'Czechia',
 'Denmark',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'Equatorial Guinea',
 'Estonia',
 'Eswatini',
 'Ethiopia',
 'Finland',
 'France',
 'Gabon',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Greenland',
 'Guatemala',
 'Guernsey',
 'Guinea',
 'Guyana',
 'Holy See',
 'Honduras',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia',
 'Iran',
 'Iraq',
 'Ireland',
 'Israel',
 'Italy',
 'Jamaica',
 'Japan',
 'Jersey',
 'Jordan',
 'Kazakhstan',
 '

In [24]:
tags

[[None, False, None, None, False, True]]

In [25]:
error

[2.3100660150125276]

In [26]:
pred

[22.385841836734695, 21.374876033057852]

In [ ]:
pred_1

In [21]:
confirm_df.head()

,Country/Region,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,1/30/20,1/31/20,2/1/20,2/2/20,2/3/20,2/4/20,2/5/20,2/6/20,2/7/20,2/8/20,2/9/20,2/10/20,2/11/20,2/12/20,2/13/20,2/14/20,2/15/20,2/16/20,2/17/20,2/18/20,2/19/20,2/20/20,2/21/20,2/22/20,2/23/20,2/24/20,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20,3/6/20,3/7/20,3/8/20,3/9/20,3/10/20,3/11/20,3/12/20,3/13/20,3/14/20,3/15/20,3/16/20,3/17/20
0,Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,4,4,5,7,7,7,11,16,21,22
1,Albania,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,10,12,23,33,38,42,51,55
2,Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,3,5,12,12,17,17,19,20,20,20,24,26,37,48,54,60
3,Andorra,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,2,39
4,Antigua and Barbuda,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1


In [55]:
data = confirm_df.iloc[69][1:].values

In [70]:
n_test = 14

In [71]:
cfg_list = exp_smoothing_configs()

In [72]:
scores = grid_search(data, cfg_list, n_test,parallel=False)
print('done')

 > Model[['add', True, None, None, False, True]] 457.924
 > Model[['add', True, None, None, False, False]] 463.594
 > Model[['add', False, None, None, False, True]] 458.607
 > Model[['add', False, None, None, False, False]] 464.303
 > Model[[None, False, None, None, False, True]] 955.264
 > Model[[None, False, None, None, False, False]] 1050.100
done


In [96]:
scores[:1][0][1]

457.92368893760215

In [89]:
for cfg, error in scores[:1]:
    print(cfg, error)

['add', True, None, None, False, True] 457.92368893760215


In [64]:
['add', True, None, None, False, True]

['add', True, None, None, False, True]

In [ ]:

if __name__ == '__main__':
	# load dataset
	series = read_csv('daily-total-female-births.csv', header=0, index_col=0)
	data = series.values
	# data split
	
	# model configs
	
	# grid search
	
	
	# list top 3 configs
	